이 페이지는 본 프로젝트의 코드를 설명하는 부분입니다. <br/>
실제 구현은 README.md에 있는 Google Drive에서 파일을 다운받아서 실행해주시기 바랍니다.

# main.py
-------------------------------

## 프로그램을 실행하는데 있어서 필요한 모듈들을 import 하는 부분입니다.
##### os 모듈은 자료를 학습시키는데에 필요한 자료를 불러오는데 쓰이는 모듈이고,argparse 모듈은 코드를 돌리는데 필요한 입력값들을 효과적으로 저장하는데에 필요한 모듈입니다.  

In [ ]:

import os # os 모듈을 가져옴
import argparse # argparse 모듈을 가져옴 
from solver import Solver  #solver모듈에서 Solver 를 가져옴
from data_loader import get_loader  # data_loader에서 get_loader를 가져옴
from torch.backends import cudnn  #torch.backends모듈에서 cudnn를 가져옴


## str을 bool type 으로 바꿔주는 함수 

In [ ]:
def str2bool(v):# str을 bool type 으로 바꿔주는 함수 
    return v.lower() in ('true') 
#입력받은 문자열 값을 소문자로 바꾼 값이 'true'의 부분집합이면 True를 아니면 False 값을 반환 

## 사용자 지정된 모듈들과 입력된 데이터로 적절한 가중치와 편향을 구하는  main 함수

In [ ]:
def main(config):
    # For fast training.
    cudnn.benchmark = True 
    # benchmark=True는 input tensor의 크기와 현재 사용중인 hardware에 따라 cudnn에서 사용하는 알고리즘을 바꿔줌.

    # Create directories if not exist. 만약 디렉토리가 없으면 필요한 디렉토리 만드는 코드
    if not os.path.exists(config.log_dir):
        os.makedirs(config.log_dir)
    if not os.path.exists(config.model_save_dir):
        os.makedirs(config.model_save_dir)
    if not os.path.exists(config.sample_dir):
        os.makedirs(config.sample_dir)
    if not os.path.exists(config.result_dir):
        os.makedirs(config.result_dir)

    # Data loader.
    celeba_loader = None  #celeba_loader변수에 None값을 저장
    rafd_loader = None  #rafd_loader변수에 None값을 저장

    # 'CelebA', 'RaFD' 중 하나의 이미지 세트를 선택했으면 선택한 이미지 세트를 get_loader함수로 컴퓨터가 인식할 수 있게하여 
    # 위에서 만든 변수에 저장하고  만약 'Both'를 선택했다면 둘 모두를 변수에 저장
    
    if config.dataset in ['CelebA', 'Both']:
        celeba_loader = get_loader(config.celeba_image_dir, config.attr_path, config.selected_attrs,
                                   'CelebA', config.mode, config.num_workers)
    if config.dataset in ['RaFD', 'Both']:
        rafd_loader = get_loader(config.rafd_image_dir, None, None,
                                 config.rafd_crop_size, config.image_size, config.batch_size,
                                 'RaFD', config.mode, config.num_workers)
    

    # Solver for training and testing StarGAN.
    solver = Solver(celeba_loader, rafd_loader, config)

    if config.mode == 'train':                             # 만약 학습모드이면 
        if config.dataset in ['CelebA', 'RaFD']:       #config.dataset에 'CelebA', 'RaFD' 중 하나가 저장되있으면
            solver.train()                                   #solver 객체에 train()함수를 실행
        elif config.dataset in ['Both']:               #config.dataset에 'Both'가 저장되있으면
            solver.train_multi()                           #solver 객체에 train_multi()함수를 실행
    
    
    elif config.mode == 'test':                           #만약 테스트 모드이면
        if config.dataset in ['CelebA', 'RaFD']:       #config.dataset에 'CelebA', 'RaFD' 중 하나가 저장되있으면
            solver.test()                                    #solver 객체에 test()함수를 실행
        elif config.dataset in ['Both']:               #config.dataset에 'Both'가 저장되있으면   
            solver.test_multi()                            #solver 객체에 test_multi()함수를 실행


## 코드의 첫시작 부분

In [ ]:
if __name__ == '__main__':# 이 부분에서 부터 시작함을 명시하는 코드
    
    parser = argparse.ArgumentParser()#argparse.ArgumentParser를 통해 parse라는 이름의 객체를 생성한다.

    # parser.add_argument를 이용하여 조건에 맞게 인자들을 입력받는다.
    
    # Model configuration. 모델에 필요한 값들을 설정 
    parser.add_argument('--c_dim', type=int, default=5, help='dimension of domain labels (1st dataset)') #1st dataset 의 차원을 설정 기본값은 5
    parser.add_argument('--c2_dim', type=int, default=8, help='dimension of domain labels (2nd dataset)')  #2nd dataset 의 차원을 설정 기본값은 5
    parser.add_argument('--celeba_crop_size', type=int, default=178, help='crop size for the CelebA dataset') #crop size for the CelebA dataset의 사이즈 조절 기본값은 128x128
    parser.add_argument('--rafd_crop_size', type=int, default=256, help='crop size for the RaFD dataset')       #crop size for the RaFD dataset의 사이즈 조절 기본값은 128x128
    parser.add_argument('--image_size', type=int, default=128, help='image resolution') # 이미지 사이즈 조절 기본값은 128x128픽셀
    parser.add_argument('--g_conv_dim', type=int, default=64, help='number of conv filters in the first layer of G')#
    parser.add_argument('--d_conv_dim', type=int, default=64, help='number of conv filters in the first layer of D')#
    parser.add_argument('--g_repeat_num', type=int, default=6, help='number of residual blocks in G')#
    parser.add_argument('--d_repeat_num', type=int, default=6, help='number of strided conv layers in D')#
    parser.add_argument('--lambda_cls', type=float, default=1, help='weight for domain classification loss')#
    parser.add_argument('--lambda_rec', type=float, default=10, help='weight for reconstruction loss')#
    parser.add_argument('--lambda_gp', type=float, default=10, help='weight for gradient penalty')# 
    
    # Training configuration. 훈련에 필요한 값들을 설정 
    parser.add_argument('--dataset', type=str, default='CelebA', choices=['CelebA', 'RaFD', 'Both']) #'중 하나를 선택해서 저장'CelebA', 'RaFD', 'Both' 기본값은 'CelebA'
    parser.add_argument('--batch_size', type=int, default=8, help='mini-batch size')# 배치사이즈 조절 기본값은 8 사이즈
    parser.add_argument('--num_iters', type=int, default=200000, help='number of total iterations for training D')
    parser.add_argument('--num_iters_decay', type=int, default=100000, help='number of iterations for decaying lr')
    parser.add_argument('--g_lr', type=float, default=0.0001, help='learning rate for G')#학습률
    parser.add_argument('--d_lr', type=float, default=0.0001, help='learning rate for D')#
    parser.add_argument('--n_critic', type=int, default=5, help='number of D updates per each G update')
    parser.add_argument('--beta1', type=float, default=0.5, help='beta1 for Adam optimizer')
    parser.add_argument('--beta2', type=float, default=0.999, help='beta2 for Adam optimizer')
    parser.add_argument('--resume_iters', type=int, default=None, help='resume training from this step')
    parser.add_argument('--selected_attrs', '--list', nargs='+', help='selected attributes for the CelebA dataset',
                        default=['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Male', 'Young'])

    # Test configuration. 테스트에 필요한 값들을 설정 
    parser.add_argument('--test_iters', type=int, default=200000, help='test model from this step') 

    # Miscellaneous.
    parser.add_argument('--num_workers', type=int, default=1)
    parser.add_argument('--mode', type=str, default='train', choices=['train', 'test'])# 학습모드인지 테스트 모드인지 선택
    parser.add_argument('--use_tensorboard', type=str2bool, default=True)          

    # Directories. 학습에 필요한 자료를 가져올 파일 이름을 설정
    parser.add_argument('--celeba_image_dir', type=str, default='data/celeba/images')
    parser.add_argument('--attr_path', type=str, default='data/celeba/list_attr_celeba.txt')
    parser.add_argument('--rafd_image_dir', type=str, default='data/RaFD/train')
    parser.add_argument('--log_dir', type=str, default='stargan/logs')
    parser.add_argument('--model_save_dir', type=str, default='stargan/models')
    parser.add_argument('--sample_dir', type=str, default='stargan/samples')
    parser.add_argument('--result_dir', type=str, default='stargan/results')

    # Step size. 스텝 사이즈
    parser.add_argument('--log_step', type=int, default=10)
    parser.add_argument('--sample_step', type=int, default=1000)
    parser.add_argument('--model_save_step', type=int, default=10000)
    parser.add_argument('--lr_update_step', type=int, default=1000)

    config = parser.parse_args()#  parser.parse_args 함수를 통해 인자들을 파싱하여 args에 저장하고 이값을 config 에 할당. 
                                #각 인자는 add_argument의 type에 지정된 형식으로 저장된다.

    print(config)                    #config 값을 프린트
    main(config)                    #config을 main 함수에 

# solver.py
--------------------------------------
solver.py는 main.py에서 사용할 다양한 함수들에 대한 선언과 구현을 하는 코드입니다.<br/>
우선 solver class에 정의된 함수들을 각각 살펴보겠습니다. (학습에 크게 영향이 없거나 설명이 불필요한 함수는 생략하였습니다)

* _init_(): 함수는 전반적인 학습을 위한 파라미터들에 대한 세팅을 해주는 함수입니다. 이는 main.py에서 사용자로부터 직접 입력받은 값을 넘겨받아서 정의해주게 됩니다. 구체적으로는 학습데이터 로딩, 영상의 여러 도메인에 대한 레이블링, 학습횟수, 학습모델, 저장 경로 등에 대한 설정입니다. 코드의 주석을 참고해주세요

In [ ]:
def __init__(self, celeba_loader, rafd_loader, config):
        """Initialize configurations."""

        # Data loader.
        self.celeba_loader = celeba_loader
        self.rafd_loader = rafd_loader

        # Model configurations. (도메인 레이블들의 차원, convolution filter의 수, residual blocks의 수 등)
        self.c_dim = config.c_dim
        self.c2_dim = config.c2_dim
        self.image_size = config.image_size
        self.g_conv_dim = config.g_conv_dim
        self.d_conv_dim = config.d_conv_dim
        self.g_repeat_num = config.g_repeat_num
        self.d_repeat_num = config.d_repeat_num
        self.lambda_cls = config.lambda_cls
        self.lambda_rec = config.lambda_rec
        self.lambda_gp = config.lambda_gp

        # Training configurations. (데이터집합, mini-batch 크기, 학습 iteration 횟수, learning rate 등)
        self.dataset = config.dataset
        self.batch_size = config.batch_size
        self.num_iters = config.num_iters
        self.num_iters_decay = config.num_iters_decay
        self.g_lr = config.g_lr
        self.d_lr = config.d_lr
        self.n_critic = config.n_critic
        self.beta1 = config.beta1
        self.beta2 = config.beta2
        self.resume_iters = config.resume_iters
        self.selected_attrs = config.selected_attrs

        # Test configurations. (학습된 모델 정하기)
        self.test_iters = config.test_iters

        # Miscellaneous. (학습에 사용되는 설정)
        self.use_tensorboard = config.use_tensorboard
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Directories. (다양한 파일의 저장 경로)
        self.log_dir = config.log_dir
        self.sample_dir = config.sample_dir
        self.model_save_dir = config.model_save_dir
        self.result_dir = config.result_dir

        # Step size. (각 항목의 update/save 빈도수)
        self.log_step = config.log_step
        self.sample_step = config.sample_step
        self.model_save_step = config.model_save_step
        self.lr_update_step = config.lr_update_step

        # Build the model and tensorboard.
        self.build_model()
        if self.use_tensorboard:
            self.build_tensorboard()

* build_model(): GAN모델에 사용되는 generator와 discriminator를 빌드해주는 함수입니다. 현재 데이터집합 별로 학습에 사용하는 도메인 차원의 수, 레이블 차원의 수 등이 다르므로 다르게 지정해줍니다. 또한 손실함수로는 Adam 함수를 사용하게됩니다.

In [ ]:
def build_model(self):
        """Create a generator and a discriminator."""
        '''generator와 discrimiator를 생성'''
        if self.dataset in ['CelebA', 'RaFD']:
            self.G = Generator(self.g_conv_dim, self.c_dim, self.g_repeat_num)
            self.D = Discriminator(self.image_size, self.d_conv_dim, self.c_dim, self.d_repeat_num) 
        elif self.dataset in ['Both']:
            self.G = Generator(self.g_conv_dim, self.c_dim+self.c2_dim+2, self.g_repeat_num)   # 2 for mask vector.
            self.D = Discriminator(self.image_size, self.d_conv_dim, self.c_dim+self.c2_dim, self.d_repeat_num)

        # 손실함수로는 Adam 함수를 사용
        self.g_optimizer = torch.optim.Adam(self.G.parameters(), self.g_lr, [self.beta1, self.beta2])
        self.d_optimizer = torch.optim.Adam(self.D.parameters(), self.d_lr, [self.beta1, self.beta2])
        self.print_network(self.G, 'G')
        self.print_network(self.D, 'D')
            
        self.G.to(self.device)
        self.D.to(self.device)

* print_network(): 네트워크의 정보를 출력해줍니다. (네트워크에 사용된 파라미터의 갯수)

In [ ]:
    def print_network(self, model, name):
        """Print out the network information."""
        '''네트워크 정보를 출력'''
        num_params = 0
        for p in model.parameters():
            num_params += p.numel()
        print(model)
        print(name)
        print("The number of parameters: {}".format(num_params))

* restore_model(): 미리 학습된 모델(generator & discriminator)를 불러온다. (가장 최근 저장된 모델을 불러온다)

In [ ]:
def restore_model(self, resume_iters):
        """Restore the trained generator and discriminator."""
        '''학습된 모델들을 가져온다'''
        print('Loading the trained models from step {}...'.format(resume_iters))
        G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(resume_iters))
        D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(resume_iters))
        self.G.load_state_dict(torch.load(G_path, map_location=lambda storage, loc: storage))
        self.D.load_state_dict(torch.load(D_path, map_location=lambda storage, loc: storage))

* build_tensorboard(): 실시간 학습 상황을 관찰하고 비교할 수 있도록 tensorboard를 구축한다. (tensorboard 구축에 관한 자세한 코드는 logger.py 파일에 있습니다.)

In [ ]:
def build_tensorboard(self):
        """Build a tensorboard logger."""
        '''tensorboard를 구축한다 (실시간 학습 상황을 보기 위함)'''
        from logger import Logger
        self.logger = Logger(self.log_dir)

* update_lr(): 초기 지정한 learning rate를 업데이트 한다. learning rate를 조절(순차적 감소)을 하는 이유는 초기에 경사하강법을 적용할때 값을 조정하는 보폭을 크게하여 빠르게 전역 최소점을 찾아가고, 점점 보폭을 작게하여 그 최소점이 계속 맴돌지 않고 올바른 최소점으로 찾아갈 수 있도록 조정해주기 위함입니다.

In [ ]:
def update_lr(self, g_lr, d_lr):
        """Decay learning rates of the generator and discriminator."""
        '''각 learning rate를 업데이트한다'''
        for param_group in self.g_optimizer.param_groups:
            param_group['lr'] = g_lr
        for param_group in self.d_optimizer.param_groups:
            param_group['lr'] = d_lr

* label2onehot(): 레이블을 one-hot vector 형식으로 변형시켜준다. one-hot vector는 중복된 데이터를 제거하고 순서대로 데이터를 저장할 수 있다는 장점이 있기때문에 이러한 작업을 통해 데이터집합을 정리하였다.

In [ ]:
def label2onehot(self, labels, dim):
        """Convert label indices to one-hot vectors."""
        '''레이블을 one-hot vector 형식으로 만들어준다'''
        batch_size = labels.size(0)
        out = torch.zeros(batch_size, dim)
        out[np.arange(batch_size), labels.long()] = 1
        return out

* create_labels(): 디버깅과 테스트를 위한 target domain label을 생성하는 함수입니다. 코드에 있는 hair color 관련 내용들은 논문 작성자들에 의하면 따로 hair color이라는 [black,blond,brown,gray] attribute를 추가하기 위하여 직접 하드코딩하였다고 합니다.

In [ ]:
def create_labels(self, c_org, c_dim=5, dataset='CelebA', selected_attrs=None):
        """Generate target domain labels for debugging and testing."""
        '''debug와 test를 위한 target domain label을 생성'''
        # Get hair color indices.
        if dataset == 'CelebA':
            hair_color_indices = []
            for i, attr_name in enumerate(selected_attrs):
                if attr_name in ['Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Gray_Hair']:
                    hair_color_indices.append(i)

        c_trg_list = []
        for i in range(c_dim):
            if dataset == 'CelebA':
                c_trg = c_org.clone()
                if i in hair_color_indices:  # Set one hair color to 1 and the rest to 0.
                    c_trg[:, i] = 1
                    for j in hair_color_indices:
                        if j != i:
                            c_trg[:, j] = 0
                else:
                    c_trg[:, i] = (c_trg[:, i] == 0)  # Reverse attribute value.
            elif dataset == 'RaFD':
                c_trg = self.label2onehot(torch.ones(c_org.size(0))*i, c_dim)

            c_trg_list.append(c_trg.to(self.device))
        return c_trg_list

* classification_loss(): 이는 확률분포 계산에 많이 사용되는 binary/softmax cross entropy loss를 사용하였다고 합니다. 코드상 구현은 특별히 어렵지 않은 것 같습니다.

In [ ]:
def classification_loss(self, logit, target, dataset='CelebA'):
        """Compute binary or softmax cross entropy loss."""
        '''확률분포 계산에 많이 사용되는 binary/softmax cross entropy loss 사용'''
        if dataset == 'CelebA':
            return F.binary_cross_entropy_with_logits(logit, target, size_average=False) / logit.size(0)
        elif dataset == 'RaFD':
            return F.cross_entropy(logit, target)

* train(): 실질적으로 본 StarGAN 모델의 학습이 이루어지는 함수입니다. 이 함수는 한가지의 데이터집합으로 학습할때 사용되고, 여러가지 데이터집합을 합쳐서 사용하게 될때는 실제 코드에 있는 train_multi() 함수를 사용하면 됩니다. 해당 함수는 굉장히 라인수가 길기 때문에 주석으로 해석을 하였습니다.

In [ ]:
def train(self):
        """Train StarGAN within a single dataset."""
        # Set data loader. 학습에 사용될 데이터집합을 선택하여 알맞은 데이터집합을 로드한다.
        if self.dataset == 'CelebA':
            data_loader = self.celeba_loader
        elif self.dataset == 'RaFD':
            data_loader = self.rafd_loader

        # Fetch fixed inputs for debugging. 디버깅을 위해서 고정된 input을 사용한다.
        data_iter = iter(data_loader)
        x_fixed, c_org = next(data_iter)
        x_fixed = x_fixed.to(self.device)
        c_fixed_list = self.create_labels(c_org, self.c_dim, self.dataset, self.selected_attrs)

        # Learning rate cache for decaying. 업데이트된 learning rate값을 실제 학습에 사용되는 learning rate 변수에 저장해준다.
        g_lr = self.g_lr
        d_lr = self.d_lr

        # Start training from scratch or resume training. 학습이 처음부터 이루어지는지, 중간에 중단한 학습을 이어서 학습할지에 따라 학습횟수를 조정하고 학습된 모델을 로딩해옵니다.
        start_iters = 0
        if self.resume_iters:
            start_iters = self.resume_iters
            self.restore_model(self.resume_iters)

        # Start training.
        print('Start training...')
        start_time = time.time()
        for i in range(start_iters, self.num_iters):

            # =================================================================================== #
            #                             1. Preprocess input data                                #
            # =================================================================================== #

            # Fetch real images and labels. 실제 영상과 그에 대한 레이블을 가져온다.
            try:
                x_real, label_org = next(data_iter)
            except:
                data_iter = iter(data_loader)
                x_real, label_org = next(data_iter)

            # Generate target domain labels randomly.
            rand_idx = torch.randperm(label_org.size(0))
            label_trg = label_org[rand_idx]

            if self.dataset == 'CelebA':
                c_org = label_org.clone()
                c_trg = label_trg.clone()
            elif self.dataset == 'RaFD':
                c_org = self.label2onehot(label_org, self.c_dim)
                c_trg = self.label2onehot(label_trg, self.c_dim)

            x_real = x_real.to(self.device)           # Input images.
            c_org = c_org.to(self.device)             # Original domain labels.
            c_trg = c_trg.to(self.device)             # Target domain labels.
            label_org = label_org.to(self.device)     # Labels for computing classification loss.
            label_trg = label_trg.to(self.device)     # Labels for computing classification loss.

            # =================================================================================== #
            #                             2. Train the discriminator                              #
            # =================================================================================== #
            # discriminator의 기본적인 역할은 generator가 생성한 fake영상을 진짜 영상인지 합성된 fake영상인지 구분하는 모델입니다.
            # 따라서 실제 영상과 fake 영상에 대한 손실량을 각각 구하고 이를 통하여 판별하게됩니다.
            
            # Compute loss with real images. 
            out_src, out_cls = self.D(x_real)
            d_loss_real = - torch.mean(out_src)
            d_loss_cls = self.classification_loss(out_cls, label_org, self.dataset)

            # Compute loss with fake images.
            x_fake = self.G(x_real, c_trg)
            out_src, out_cls = self.D(x_fake.detach())
            d_loss_fake = torch.mean(out_src)

            # Compute loss for gradient penalty.
            alpha = torch.rand(x_real.size(0), 1, 1, 1).to(self.device)
            x_hat = (alpha * x_real.data + (1 - alpha) * x_fake.data).requires_grad_(True)
            out_src, _ = self.D(x_hat)
            d_loss_gp = self.gradient_penalty(out_src, x_hat)

            # Backward and optimize.
            d_loss = d_loss_real + d_loss_fake + self.lambda_cls * d_loss_cls + self.lambda_gp * d_loss_gp
            self.reset_grad()
            d_loss.backward()
            self.d_optimizer.step()

            # Logging.
            loss = {}
            loss['D/loss_real'] = d_loss_real.item()
            loss['D/loss_fake'] = d_loss_fake.item()
            loss['D/loss_cls'] = d_loss_cls.item()
            loss['D/loss_gp'] = d_loss_gp.item()
            
            # =================================================================================== #
            #                               3. Train the generator                                #
            # =================================================================================== #
            
            if (i+1) % self.n_critic == 0:
                # Original-to-target domain.
                x_fake = self.G(x_real, c_trg)
                out_src, out_cls = self.D(x_fake)
                g_loss_fake = - torch.mean(out_src)
                g_loss_cls = self.classification_loss(out_cls, label_trg, self.dataset)

                # Target-to-original domain.
                x_reconst = self.G(x_fake, c_org)
                g_loss_rec = torch.mean(torch.abs(x_real - x_reconst))

                # Backward and optimize.
                g_loss = g_loss_fake + self.lambda_rec * g_loss_rec + self.lambda_cls * g_loss_cls
                self.reset_grad()
                g_loss.backward()
                self.g_optimizer.step()

                # Logging.
                loss['G/loss_fake'] = g_loss_fake.item()
                loss['G/loss_rec'] = g_loss_rec.item()
                loss['G/loss_cls'] = g_loss_cls.item()

            # =================================================================================== #
            #                                 4. Miscellaneous (기타)                              #
            # =================================================================================== #

            # Print out training information.
            if (i+1) % self.log_step == 0:
                et = time.time() - start_time
                et = str(datetime.timedelta(seconds=et))[:-7]
                log = "Elapsed [{}], Iteration [{}/{}]".format(et, i+1, self.num_iters)
                for tag, value in loss.items():
                    log += ", {}: {:.4f}".format(tag, value)
                print(log)

                if self.use_tensorboard:
                    for tag, value in loss.items():
                        self.logger.scalar_summary(tag, value, i+1)

            # Translate fixed images for debugging. 초기에 고정해준 영상을 사용하여 디버깅을 진행한다.
            if (i+1) % self.sample_step == 0:
                with torch.no_grad():
                    x_fake_list = [x_fixed]
                    for c_fixed in c_fixed_list:
                        x_fake_list.append(self.G(x_fixed, c_fixed))
                    x_concat = torch.cat(x_fake_list, dim=3)
                    sample_path = os.path.join(self.sample_dir, '{}-images.jpg'.format(i+1))
                    save_image(self.denorm(x_concat.data.cpu()), sample_path, nrow=1, padding=0)
                    print('Saved real and fake images into {}...'.format(sample_path))

            # Save model checkpoints. (정해진 학습횟수 단위로 모델을 저장한다.)
            if (i+1) % self.model_save_step == 0:
                G_path = os.path.join(self.model_save_dir, '{}-G.ckpt'.format(i+1))
                D_path = os.path.join(self.model_save_dir, '{}-D.ckpt'.format(i+1))
                torch.save(self.G.state_dict(), G_path)
                torch.save(self.D.state_dict(), D_path)
                print('Saved model checkpoints into {}...'.format(self.model_save_dir))

            # Decay learning rates. learning rate를 조정해준다 (순차적으로 감소시킴)
            if (i+1) % self.lr_update_step == 0 and (i+1) > (self.num_iters - self.num_iters_decay):
                g_lr -= (self.g_lr / float(self.num_iters_decay))
                d_lr -= (self.d_lr / float(self.num_iters_decay))
                self.update_lr(g_lr, d_lr)
                print ('Decayed learning rates, g_lr: {}, d_lr: {}.'.format(g_lr, d_lr))

* test(): 이 함수는 학습된 모델을 사용하여 실제 사용자가 원하는 사진에 대해 성공적으로 일을 수행하는지 test할때 사용되는 함수입니다. 이 함수는 한가지의 데이터집합으로 test할때 사용되고, 여러가지 데이터집합을 합쳐서 사용하게 될때는 실제 코드에 있는 test_multi() 함수를 사용하면 됩니다.

In [ ]:
def test(self):
        """Translate images using StarGAN trained on a single dataset."""
        # Load the trained generator. test는 기존에 학습된 모델을 사용하여 진행하기 때문에 학습된 모델을 불러온다.
        self.restore_model(self.test_iters)
        
        # Set data loader. test에 사용될 데이터집합을 로딩한다.
        if self.dataset == 'CelebA':
            data_loader = self.celeba_loader
        elif self.dataset == 'RaFD':
            data_loader = self.rafd_loader
        
        with torch.no_grad():
            for i, (x_real, c_org) in enumerate(data_loader):

                # Prepare input images and target domain labels.
                x_real = x_real.to(self.device)
                c_trg_list = self.create_labels(c_org, self.c_dim, self.dataset, self.selected_attrs)

                # Translate images. 학습된 weight들로 영상 변환을 한다.
                x_fake_list = [x_real]
                for c_trg in c_trg_list:
                    x_fake_list.append(self.G(x_real, c_trg))

                # Save the translated images.
                x_concat = torch.cat(x_fake_list, dim=3)
                result_path = os.path.join(self.result_dir, '{}-images.jpg'.format(i+1))
                save_image(self.denorm(x_concat.data.cpu()), result_path, nrow=1, padding=0)
                print('Saved real and fake images into {}...'.format(result_path))

# logger.py
----------------------------------
logger.py는 tensorboard를 구축해주는 파일입니다. tensorboard는 학습 과정의 로그를 저장하여 웹으로 띄워서 사용자가 학습 과정과 결과에 대한 비교를 실시간으로 모니터링 하며 확인할 수 있도록 만들어줍니다.

In [3]:
import tensorflow as tf


class Logger(object):
    """Tensorboard logger."""

    def __init__(self, log_dir):
        """Initialize summary writer."""
        self.writer = tf.summary.create_file_writer(log_dir)

    def scalar_summary(self, tag, value, step):
        """Add scalar summary."""
        summary = tf.Summary(value=[tf.Summary.Value(tag=tag, simple_value=value)])
        self.writer.add_summary(summary, step)


# model.py

### 라이브러리 불러오기 

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

## Residual Block with instance normalization 함수

In [ ]:
class ResidualBlock(nn.Module):
    def __init__(self, dim_in, dim_out):
        super(ResidualBlock, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(dim_in, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),
            nn.Conv2d(dim_out, dim_out, kernel_size=3, stride=1, padding=1, bias=False),
            nn.InstanceNorm2d(dim_out, affine=True, track_running_stats=True))
        
    def forward(self, x):
        return x + self.main(x)

* nn.sequential란  여러가지 nn.module를 한 컨테이너에 집어넣고 한번에 돌리는 방법이다.
* nn.conv2d : (입력값,필터값,strides,padding)
* 필터값을 각각 행렬로 정의해주고, padding인자를 통해 padding사이즈를 정한다.
* ReLU : rectified Linear unit의 약자로 활성화 함수이다.
* ReLU는 sigmoid function의 문제인 gradient vanishing을 해결하기위해 생겨났다.

## GAN 의 생성자(generator)함수

* 생성자는 랜덤 벡터를 입력으로 받아 가짜 이미지를 출력한다. 

In [ ]:
class Generator(nn.Module):
    
    #네트워크 구조
    def __init__(self, conv_dim=64, c_dim=5, repeat_num=6):
        super(Generator, self).__init__()

        layers = []
        layers.append(nn.Conv2d(3+c_dim, conv_dim, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.InstanceNorm2d(conv_dim, affine=True, track_running_stats=True))
        layers.append(nn.ReLU(inplace=True))

        # Down-sampling layers.
        curr_dim = conv_dim
        for i in range(2):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim*2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True))
            curr_dim = curr_dim * 2

        # Bottleneck layers.
        for i in range(repeat_num):
            layers.append(ResidualBlock(dim_in=curr_dim, dim_out=curr_dim))

        # Up-sampling layers.
        for i in range(2):
            layers.append(nn.ConvTranspose2d(curr_dim, curr_dim//2, kernel_size=4, stride=2, padding=1, bias=False))
            layers.append(nn.InstanceNorm2d(curr_dim//2, affine=True, track_running_stats=True))
            layers.append(nn.ReLU(inplace=True))
            curr_dim = curr_dim // 2

        layers.append(nn.Conv2d(curr_dim, 3, kernel_size=7, stride=1, padding=3, bias=False))
        layers.append(nn.Tanh())
        self.main = nn.Sequential(*layers)
    def forward(self, x, c):
        # Replicate spatially and concatenate domain information.
        # Note that this type of label conditioning does not work at all if we use reflection padding in Conv2d.
        # This is because instance normalization ignores the shifting (or bias) effect.
        c = c.view(c.size(0), c.size(1), 1, 1)
        c = c.repeat(1, 1, x.size(2), x.size(3))
        x = torch.cat([x, c], dim=1)
        return self.main(x)

* torch.cat : 다른 길이의 텐서를 하나로 묶을 때 사용한다.
* view : numpy의 reshape와 유사하다.

## GAN의 구분자(Discriminator)함수

* 구분자는 이미지를 입력으로 받아 이미지가 진짜인지 가짜인지 출력한다.

In [ ]:
class Discriminator(nn.Module):
    """Discriminator network with PatchGAN."""
    #네트워크 구조 
    def __init__(self, image_size=128, conv_dim=64, c_dim=5, repeat_num=6):
        super(Discriminator, self).__init__()
        layers = []
        layers.append(nn.Conv2d(3, conv_dim, kernel_size=4, stride=2, padding=1))
        layers.append(nn.LeakyReLU(0.01))

        curr_dim = conv_dim
        for i in range(1, repeat_num):
            layers.append(nn.Conv2d(curr_dim, curr_dim*2, kernel_size=4, stride=2, padding=1))
            layers.append(nn.LeakyReLU(0.01))
            curr_dim = curr_dim * 2

        kernel_size = int(image_size / np.power(2, repeat_num))
        self.main = nn.Sequential(*layers)
        # convolution은 2차원 배열로 설정한다.
        self.conv1 = nn.Conv2d(curr_dim, 1, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(curr_dim, c_dim, kernel_size=kernel_size, bias=False)

    def forward(self, x):
        h = self.main(x)
        out_src = self.conv1(h)
        out_cls = self.conv2(h)
        return out_src, out_cls.view(out_cls.size(0), out_cls.size(1))


* LeakyReLU는 각 뉴런의 출력값이 0보다 높으면 그대로 놔두고, 0보다 낮으면 정해진 작은 숫자를 곱하는 간단한 함수다.

# data_loader.py

### 라이브러리 및 데이터 불러오기

In [ ]:
from torch.utils import data
from torchvision import transforms as T
from torchvision.datasets import ImageFolder
from PIL import Image
import torch
import os
import random

####  사람 얼굴 이미지 오픈데이터셋 (CelebA)

In [ ]:
class CelebA(data.Dataset):
    
    #celebA dataset 의 초기화
    def __init__(self, image_dir, attr_path, selected_attrs, transform, mode):
        """Initialize and preprocess the CelebA dataset."""
        self.image_dir = image_dir
        self.attr_path = attr_path
        self.selected_attrs = selected_attrs
        self.transform = transform
        self.mode = mode
        self.train_dataset = []
        self.test_dataset = []
        self.attr2idx = {}
        self.idx2attr = {}
        self.preprocess()

        if mode == 'train':
            self.num_images = len(self.train_dataset)
        else:
            self.num_images = len(self.test_dataset)

#### celebA  의 전처리 함수 

In [ ]:
def preprocess(self):
        """Preprocess the CelebA attribute file."""
        lines = [line.rstrip() for line in open(self.attr_path, 'r')]
        all_attr_names = lines[1].split()
        for i, attr_name in enumerate(all_attr_names):
            self.attr2idx[attr_name] = i
            self.idx2attr[i] = attr_name

        lines = lines[2:]
        random.seed(1234)
        random.shuffle(lines)

        for i, line in enumerate(lines):
            split = line.split()
            filename = split[0]
            values = split[1:]

            label = []
            for attr_name in self.selected_attrs:
                idx = self.attr2idx[attr_name]
                label.append(values[idx] == '1')

            if (i+1) < 2000:
                self.test_dataset.append([filename, label])
            else:
                self.train_dataset.append([filename, label])

        print('Finished preprocessing the CelebA dataset...')

* random.shuffle(): random모듈을 이용한 편의함수로 리턴값이 없고, 입력 시퀸스를 직접 바꾸는 in-place 연산이다.

#### 이미지를 불러오는 함수

In [ ]:
def __getitem__(self, index):
        """Return one image and its corresponding attribute label."""
        dataset = self.train_dataset if self.mode == 'train' else self.test_dataset
        filename, label = dataset[index]
        image = Image.open(os.path.join(self.image_dir, filename))
        return self.
    form(image), torch.FloatTensor(label)

#### 이미지의 넘버 확인

In [ ]:
   def __len__(self):
        """Return the number of images."""
        return self.num_images

#### data를 로드한다.

In [ ]:

def get_loader(image_dir, attr_path, selected_attrs, crop_size=178, image_size=128, 
               batch_size=16, dataset='CelebA', mode='train', num_workers=1):
    """Build and return a data loader."""
    transform = []
    if mode == 'train':
        transform.append(T.RandomHorizontalFlip())
    transform.append(T.CenterCrop(crop_size))
    transform.append(T.Resize(image_size))
    transform.append(T.ToTensor())
    transform.append(T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5)))
    transform = T.Compose(transform)

    if dataset == 'CelebA':
        dataset = CelebA(image_dir, attr_path, selected_attrs, transform, mode)
    elif dataset == 'RaFD':
        dataset = ImageFolder(image_dir, transform)

    data_loader = data.DataLoader(dataset=dataset,
                                  batch_size=batch_size,
                                  shuffle=(mode=='train'),
                                  num_workers=num_workers)
    return data_loader